In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'/Users/physics/nanogw/Optimal-Statisitic-iPTA-Summer-2022')

In [2]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import numpy as np
import pickle
import json
import glob
import os

import matplotlib.pyplot as plt
%matplotlib inline


from enterprise.signals import signal_base
from enterprise.signals import gp_signals
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import gp_priors

from enterprise_extensions import model_utils, blocks
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise.signals import selections
import enterprise_extensions.chromatic.chromatic


from enterprise_warp import enterprise_warp

import h5py
import la_forge.core as co
import ppta_dr2_models as dr2models

In [3]:
noisefile = '/Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [4]:
with open('/Users/physics/nanogw/data/ppta_dr2.pkl','rb') as fin:
    psrs=pickle.load(fin)

In [5]:
for psr in psrs: 
    print(psr)

In [6]:
psrsname=[]
for p in psrs:
    psrsname.append(p.name)

print(psrsname)

['J0437-4715', 'J0613-0200', 'J0711-6830', 'J1017-7156', 'J1022+1001', 'J1024-0719', 'J1045-4509', 'J1125-6014', 'J1446-4701', 'J1545-4550', 'J1600-3053', 'J1603-7202', 'J1643-1224', 'J1713+0747', 'J1730-2304', 'J1732-5049', 'J1744-1134', 'J1824-2452A', 'J1832-0836', 'J1857+0943', 'J1909-3744', 'J1939+2134', 'J2124-3358', 'J2129-5721', 'J2145-0750', 'J2241-5236']


In [ ]:
"""
import os
 
# Function to rename multiple files
folder = "/Users/physics/Desktop/paramfilespta/"

p=0

for count, filename in enumerate(sorted(os.listdir(folder))):
    name=psrsname[count]
    add=name
    old=filename
    oldpath=folder+filename
    new=add+'_'+filename[5:] 
    newpath=folder+new
    print(newpath)
         
    # rename() function will
    # rename all the files
    os.rename(oldpath, newpath)
"""

In [ ]:
"""
keys = []
values = []
for k, v in noisedict.items():
    if 'J2241' in k: 
        keys.append(k)
        values.append(noisedict[k])
pulsar_noise=dict(zip(keys, values))
"""

In [ ]:
'''
with open("/Users/physics/Desktop/Noisepta/J2241noisefile.json", "w") as outfile:
    json.dump(pulsar_noise, outfile)
'''

In [7]:
class opts(object):pass
setattr(opts,'extra_model_terms','None')
setattr(opts,'job_config_xlsx',None)
setattr(opts,'num',0)
setattr(opts,'mpi_regime',None)
setattr(opts,'wipe_old_output',None)

In [8]:
len(psrs)

26

In [9]:
full_model=[]
models=[]
paramspta = enterprise_warp.enterprise_warp.Params('/Users/physics/Desktop/ppta_gw/pta.txt', opts=opts)
pulsars = dr2models.PPTADR2Models(psrs,paramspta)
model= pulsars.gwb()

------------------
Setting default parameters with file  /Users/physics/Desktop/ppta_gw/pta.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J0437-4715.par
Current .tim file:  J0437-4715.tim
Number of Fourier frequencies for the GWB/CPL signal:  91
Adding HD ORF


In [ ]:
noisepath="/Users/physics/Desktop/Noisemodelpta/"
parampath="/Users/physics/Desktop/paramfilespta/"
full_model=[]
models=[]
point=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]

params = enterprise_warp.enterprise_warp.Params('/Users/physics/Desktop/ppta_gw/pta.txt', opts=opts)
pulsars = dr2models.PPTADR2Models(psrs,params)

for p in point:
    with open('/Users/physics/Desktop/ppta_gw/pta.json','r') as f:
        model_names = json.load(f)[psrs[p].name]
    model= pulsars.gwb()
    model += pulsars.efac()
    for ky in model_names.keys():
        print(ky)
        if ky != 'efac':
            if ky in ['system_noise','paired_ppta_band_noise','ppta_band_noise','fd_sys_g']:
                model +=getattr(pulsars,ky)(option=model_names[ky])
            else: 
                model += getattr(pulsars,ky)()
                
models.append(model)
full_model.append(model(psr))

In [10]:
noisepath="/Users/physics/Desktop/Noisemodelpta/"
parampath="/Users/physics/Desktop/paramfilespta/"
point=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
full_model=[]
models=[]

paramspta = enterprise_warp.enterprise_warp.Params('/Users/physics/Desktop/ppta_gw/pta.txt', opts=opts)
pulsars = dr2models.PPTADR2Models(psrs,paramspta)

for psr,param_file,noise_file,n in zip(psrs,sorted(os.listdir("/Users/physics/Desktop/paramfilespta/")),sorted(os.listdir("/Users/physics/Desktop/Noisemodelpta")),point): 
    model= pulsars.gwb()
    pnm = psr.name
    print(noisepath+noise_file)
    with open(noisepath+noise_file,'r') as f:
        model_names = json.load(f)[pnm]
    opts.num=n
    print(parampath+param_file)
    params = enterprise_warp.enterprise_warp.Params(parampath+param_file, opts=opts)
    pulsar = dr2models.PPTADR2Models(psr,params)
    model += pulsar.efac()
    for ky in model_names.keys():
        print(ky)
        if ky != 'efac':
            if ky in ['system_noise','paired_ppta_band_noise','ppta_band_noise','fd_sys_g']:
                model +=getattr(pulsar,ky)(option=model_names[ky])
            else: 
                model += getattr(pulsar,ky)()
                
    models.append(model)
        
    #with open(f'path/to/pickled/models/{pnm}_model_psr.pkl','wb') as fout:
     #   pickle.dump(full_model,fout)

------------------
Setting default parameters with file  /Users/physics/Desktop/ppta_gw/pta.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J0437-4715.par
Current .tim file:  J0437-4715.tim
Number of Fourier frequencies for the GWB/CPL signal:  91
Adding HD ORF
/Users/physics/Desktop/Noisemodelpta/J0437-4715_noisemodel.json
/Users/physics/Desktop/paramfilespta/J0437-4715_param_fixed_whitenoise.txt
------------------
Setting default parameters with file  /Users/physics/Desktop/paramfilespta/J0437-4715_param_fixed_whitenoise.txt
Setting default Solar System Epheme

Current .par file:  J1446-4701.par
Current .tim file:  J1446-4701.tim
efac
equad
spin_noise
dm_noise
Number of Fourier frequencies for the GWB/CPL signal:  91
Adding HD ORF
/Users/physics/Desktop/Noisemodelpta/J1545-4550_noisemodel.json
/Users/physics/Desktop/paramfilespta/J1545-4550_param_fixed_whitenoise.txt
------------------
Setting default parameters with file  /Users/physics/Desktop/paramfilespta/J1545-4550_param_fixed_whitenoise.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J1545-4550.par
Current .tim file:  J1545-4550.tim
efac
equad
spin_noise
dm_noise

Current .par file:  J1824-2452A.par
Current .tim file:  J1824-2452A.tim
efac
equad
spin_noise
dm_noise
paired_ppta_band_noise
Number of Fourier frequencies for the GWB/CPL signal:  91
Adding HD ORF
/Users/physics/Desktop/Noisemodelpta/J1832-0836_noisemodel.json
/Users/physics/Desktop/paramfilespta/J1832-0836_param_fixed_whitenoise.txt
------------------
Setting default parameters with file  /Users/physics/Desktop/paramfilespta/J1832-0836_param_fixed_whitenoise.txt
Setting default Solar System Ephemeris: DE436
Setting a default Enterprise clock convention (check the code)
Using all available pulsars from .par/.tim directory
Setting a default linear timing model
Including transient events to specific pulsar models
Setting reference radio frequency to 1400 MHz
------------------
Setting sampler kwargs from the parameter file:
Setting SCAMweight to 30
Setting AMweight to 15
Setting DEweight to 50
------------------
Current .par file:  J1832-0836.par
Current .tim file:  J1832-0836.tim
efac


In [ ]:
noisepath="/Users/physics/Desktop/Noisemodelpta/"
parampath="/Users/physics/Desktop/paramfilespta/"
point=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
full_model=[]
models=[]

Tspan = model_utils.get_tspan(psrs)


for psr,param_file,noise_file,n in zip(psrs,sorted(os.listdir("/Users/physics/Desktop/paramfilespta/")),sorted(os.listdir("/Users/physics/Desktop/Noisemodelpta")),point): 
    pnm = psr.name
    print(noisepath+noise_file)
    with open(noisepath+noise_file,'r') as f:
        model_names = json.load(f)[pnm]
    opts.num=n
    print(parampath+param_file)
    params = enterprise_warp.enterprise_warp.Params(parampath+param_file, opts=opts)
    pulsar = dr2models.PPTADR2Models(psr,params)
    model= blocks.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, components=91, gamma_val=4.33, name='gw', orf='hd')
    model += pulsar.efac()
    for ky in model_names.keys():
        print(ky)
        if ky != 'efac':
            if ky in ['system_noise','paired_ppta_band_noise','ppta_band_noise','fd_sys_g']:
                model +=getattr(pulsar,ky)(option=model_names[ky])
            else: 
                model += getattr(pulsar,ky)()
                
    models.append(model)
        
    #with open(f'path/to/pickled/models/{pnm}_model_psr.pkl','wb') as fout:
     #   pickle.dump(full_model,fout)

In [ ]:
#import cloudpickle
#with open('/Users/physics/nanogw/data/ppta_models.pkl','wb') as fout:
 #   cloudpickle.dump(models,fout)

In [11]:
for p in psrs:
    p.name=p.name+'_ppta'
for p in psrs:
    print(p.name)

J0437-4715_ppta
J0613-0200_ppta
J0711-6830_ppta
J1017-7156_ppta
J1022+1001_ppta
J1024-0719_ppta
J1045-4509_ppta
J1125-6014_ppta
J1446-4701_ppta
J1545-4550_ppta
J1600-3053_ppta
J1603-7202_ppta
J1643-1224_ppta
J1713+0747_ppta
J1730-2304_ppta
J1732-5049_ppta
J1744-1134_ppta
J1824-2452A_ppta
J1832-0836_ppta
J1857+0943_ppta
J1909-3744_ppta
J1939+2134_ppta
J2124-3358_ppta
J2129-5721_ppta
J2145-0750_ppta
J2241-5236_ppta


In [12]:
len(models)

26

In [13]:
len(psrs)

26

In [14]:
full_model2=[]
for m,psr in zip(models,psrs):
    full_model2.append(m(psr))

In [15]:
noisefile = '/Users/physics/nanogw/data/ppta_dr2_TN_noise.json'

with open(noisefile, 'r') as f:
    noisedict = json.load(f)

In [16]:
noisedictcp=noisedict.copy()
#for ky,val in noisedict.items():
#    if 'equad'in ky and not 'tnequad' in ky:
#        kynew=ky.replace('equad','tnequad')
#        noisedictcp[kynew] = noisedict[ky]

noisedictcp2=noisedictcp.copy()
for ky,val in noisedictcp.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    noisedictcp2[kynew] = noisedictcp[ky]
for ky,val in noisedictcp.items():
    if ky in noisedictcp:
        noisedictcp2.pop(ky)

In [17]:
pta = signal_base.PTA(full_model2)
# We need to set the white noise parameters to the values in the noise dictionary
pta.set_default_params(noisedictcp2)

In [18]:
print(pta.summary())

enterprise v3.3.1,  Python v3.9.13

Signal Name                              Signal Class                   no. Parameters      
J0437-4715_ppta_gw                       FourierBasisCommonGP           2                   

params:
gw_log10_A:Uniform(pmin=-20.0, pmax=-6.0)                                                 
gw_gamma:Uniform(pmin=0.0, pmax=10.0)                                                     
__________________________________________________________________________________________
J0437-4715_ppta_measurement_noise        MeasurementNoise               0                   

params:
J0437-4715_ppta_CASPSR_40CM_efac:Constant=0.7430934464857502                              
J0437-4715_ppta_CPSR2_10CM_efac:Constant=1.1394645851791254                               
J0437-4715_ppta_CPSR2_20CM_efac:Constant=2.3071643789004312                               
J0437-4715_ppta_CPSR2_50CM_efac:Constant=1.120916497680611                                
J0437-4715_ppta_PDFB1_10CM_efa

In [19]:
ostat = opt_stat.OptimalStatistic(psrs, pta=pta, orf='hd')
ostat_dip = opt_stat.OptimalStatistic(psrs, pta=pta, orf='dipole')
ostat_mono = opt_stat.OptimalStatistic(psrs, pta=pta, orf='monopole')

In [20]:
# Load up the maximum-likelihood values for the pulsars' red noise parameters and the common red process
# These values come from the results of a Bayesian search

with open("/Users/physics/nanogw/data/ppta_ml_params.pkl",'rb') as fin:
    ml_params=pickle.load(fin)

ml_paramscp=ml_params.copy()
for ky,val in ml_params.items():
    split=ky.split('_')
    split.insert(1,'ppta')
    kynew='_'.join(split)
    ml_paramscp[kynew] = ml_params[ky]
for ky,val in ml_params.items():
    if ky in ml_params:
        ml_paramscp.pop(ky)
ml_paramscp['gw_log10_A']=ml_paramscp['gw_ppta_log10_A']

In [21]:
ml_paramscp['J0437-4715_ppta_chromatic_gp_idx']=4
ml_paramscp['J0613-0200_ppta_chromatic_gp_idx']=4
ml_paramscp['J1017-7156_ppta_chromatic_gp_idx']=2.39
ml_paramscp['J1045-4509_ppta_chromatic_gp_idx']=1.82
ml_paramscp['J1939+2134_ppta_chromatic_gp_idx']=4

In [22]:
ml_paramscp

{'J0437-4715_ppta_band_noise_7_20CM_gamma': 3.049994802210053,
 'J0437-4715_ppta_band_noise_7_20CM_log10_A': -13.650642385600916,
 'J0437-4715_ppta_band_noise_by_B_4050CM_4050CM_gamma': 1.2868547453651276,
 'J0437-4715_ppta_band_noise_by_B_4050CM_4050CM_log10_A': -13.503948657095098,
 'J0437-4715_ppta_chromatic_gp_gamma': 2.065814974203881,
 'J0437-4715_ppta_chromatic_gp_log10_A': -14.304840939440638,
 'J0437-4715_ppta_dm_gp_gamma': 3.2914690295091478,
 'J0437-4715_ppta_dm_gp_log10_A': -13.587604509820993,
 'J0437-4715_ppta_dmexp_idx': -0.27328321409228395,
 'J0437-4715_ppta_dmexp_log10_Amp': -6.184373208608071,
 'J0437-4715_ppta_dmexp_log10_tau': 1.4560385098541047,
 'J0437-4715_ppta_dmexp_t0': 57061.88085496656,
 'J0437-4715_ppta_fd1_sys_CASPSR_40CM_CASPSR_40CM_slope': 3.92553089435287e-08,
 'J0437-4715_ppta_fd1_sys_CPSR2_50CM_CPSR2_50CM_slope': 9.9817303902494e-08,
 'J0437-4715_ppta_fd1_sys_PDFB1_10CM_PDFB1_10CM_slope': 3.966358356544e-10,
 'J0437-4715_ppta_fd1_sys_PDFB1_1433_PDFB1_

In [23]:
for ky in ml_paramscp.keys():
    if ky not in pta.param_names:
        print(ky)

J1045-4509_ppta_red_noise_gamma
J1045-4509_ppta_red_noise_log10_A
J1125-6014_ppta_red_noise_gamma
J1125-6014_ppta_red_noise_log10_A
gw_ppta_log10_A
nmodel_ppta
lnpost_ppta
lnlike_ppta
chain_ppta_accept
pt_ppta_chain_accept


In [24]:
for nm in pta.param_names:
    if nm not in ml_paramscp.keys():
        print(nm)

gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma
gw_gamma


In [25]:
# Compute the optimal statistic
# The optimal statistic returns five quantities:
#  - xi: an array of the angular separations between the pulsar pairs (in radians)
#  - rho: an array of the cross-correlations between the pulsar pairs
#  - sig: an array of the uncertainty in the cross-correlations
#  - OS: the value of the optimal statistic
#  - OS_sig: the uncertainty in the optimal statistic

xi, rho, sig, OS, OS_sig = ostat.compute_os(params=ml_paramscp) #HD
print(OS, OS_sig, OS/OS_sig)

_, _, _, OS_dip, OS_sig_dip = ostat_dip.compute_os(params=ml_paramscp) #Dipole
print(OS_dip, OS_sig_dip, OS_dip/OS_sig_dip)

_, _, _, OS_mono, OS_sig_mono = ostat_mono.compute_os(params=ml_paramscp) #Monopole 
print(OS_mono, OS_sig_mono, OS_mono/OS_sig_mono)

/Users/physics/opt/anaconda3/envs/nanogw/lib/python3.9/site-packages/enterprise_extensions/frequentist/optimal_statistic.py:113: UserWarning: gw_gamma is not included in the parameter dictionary. Drawing a random value.


8.655564660931205e-29 2.882772373557218e-30 30.025140868998292
-1.160881949984826e-29 7.371258060286973e-31 -15.7487628365521
1.6377945430326404e-30 4.2967247829362025e-31 3.81172782938506


In [26]:
pta.get_lnlikelihood(ml_paramscp)

KernelMatrix(1819133.34155503)

In [27]:
ml_params['lnlike']

1895529.812548

In [ ]:
pta.get_lnprior(ml_paramscp)

In [ ]:
Tspan